In [1]:
from fastai.text import *
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm
import re
import pdb
import random
from fastai.imports import *
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, matthews_corrcoef, f1_score


In [2]:
import fastai, torch, fastprogress
fastai.__version__ , torch.__version__, fastprogress.__version__

('1.0.60', '1.7.1', '0.2.7')

In [5]:
torch.cuda.set_device(0)

In [6]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [7]:
random_seed(42, True)

In [9]:
path = Path('./')

In [ ]:
BASE_DIR = "home/temp/data/eacl/ta/"

In [ ]:
df_train = pd.read_csv(f"{BASE_DIR}/tamil_offensive_full_train_transliterated.csv")
df_train.dropna(inplace=True)
# df_train.head()

In [11]:
df_train.shape

(35139, 2)

In [12]:
Counter(df_train['label'])

Counter({0: 25425, 4: 1454, 5: 454, 3: 2557, 1: 2906, 2: 2343})

In [13]:
df_valid = pd.read_csv(f"{BASE_DIR}/temp/data/eacl/ta/tamil_offensive_full_dev_transliterated.csv")
df_valid.dropna(inplace=True)
df_valid.head()

,sentence,label
0,Handsome hunk keri vaa thalaivaa,0
1,thenkaachi maavattam naataar chamuthaayam chaa...,0
2,je vous aime bravo pour clip de merde que j éc...,4
3,chirappu melum ithu poonra pataippukal mika av...,0
4,Vera level BGM ..semma trailer. 🤞,0


In [15]:
df_test = pd.read_csv(f"{BASE_DIR}/temp/data/eacl/ta/tamil_offensive_full_test_transliterated.csv")
df_test.dropna(inplace=True)
df_test.head()

,sentence
0,14.12.2018epo trailer pathutu irken ...Semay...
1,Paka thana poro movie la Enna irukunu
2,“U kena tunggu lebih lama lagi untuk tahu saya...
3,Suriya anna vera level anna mass
4,suma kaththaatha da sound over a pooda kudaath...


In [31]:
df_train.shape, df_valid.shape, df_test.shape

((35139, 2), (4388, 2), (4392, 1))

In [32]:
Counter(df_train['label']), Counter(df_valid['label'])

(Counter({0: 25425, 4: 1454, 5: 454, 3: 2557, 1: 2906, 2: 2343}),
 Counter({0: 3193, 4: 172, 3: 295, 2: 307, 1: 356, 5: 65}))

In [14]:
label_cols = ['label']
text_cols = ['sentence']

### Tokenization + Data Cleaning

In [15]:
def handle_all_caps(t: str) -> str:
    tokens = t.split()
    tokens = replace_all_caps(tokens)
    return ' '.join(tokens)

def handle_upper_case_first_letter(t: str) -> str:
    tokens = t.split()
    tokens = deal_caps(tokens)
    return ' '.join(tokens)

def lower_case_everything(t: str) -> str:
    return t.lower().replace('@user', '').replace('#tag ', '').replace('rt ', '').strip()

In [16]:
class CodeMixedTamilTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str(path/"../models/taen_spm.model"))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

In [17]:
sp = spm.SentencePieceProcessor()
sp.Load(str(path/"../models/taen_spm.model"))
itos = [sp.IdToPiece(int(i)) for i in range(8000)]

In [23]:
itos[:20]

['xxunk',
 'xxbos',
 'xxeos',
 'xxpad',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 '.',
 ',',
 '▁',
 's',
 'a',
 '="',
 'in',
 'doc',
 't',
 'il',
 'i']

In [24]:
# 8,000 is the vocab size that we chose in sentencepiece
taen_vocab = Vocab(itos)


In [19]:
tokenizer = Tokenizer(lang='taen', tok_func=CodeMixedTamilTokenizer)

In [20]:
tokenizer.pre_rules.append(lower_case_everything)
tokenizer.pre_rules.append(handle_all_caps)
tokenizer.pre_rules.append(handle_upper_case_first_letter)

In [21]:
tokenizer.special_cases, tokenizer.pre_rules, tokenizer.post_rules

(['xxunk',
  'xxpad',
  'xxbos',
  'xxeos',
  'xxfld',
  'xxmaj',
  'xxup',
  'xxrep',
  'xxwrep'],
 [<function fastai.text.transform.fix_html(x: str) -> str>,
  <function fastai.text.transform.replace_rep(t: str) -> str>,
  <function fastai.text.transform.replace_wrep(t: str) -> str>,
  <function fastai.text.transform.spec_add_spaces(t: str) -> str>,
  <function fastai.text.transform.rm_useless_spaces(t: str) -> str>,
  <function __main__.lower_case_everything(t: str) -> str>,
  <function __main__.handle_all_caps(t: str) -> str>,
  <function __main__.handle_upper_case_first_letter(t: str) -> str>],
 [<function fastai.text.transform.replace_all_caps(x: Collection[str]) -> Collection[str]>,
  <function fastai.text.transform.deal_caps(x: Collection[str]) -> Collection[str]>])

In [22]:
tokens = tokenizer.process_all(['epo trailer pathutu irken '])
''.join(tokens[0]), tokens

('▁epo▁trailer▁pathutu▁irken',
 [['▁e', 'po', '▁tra', 'il', 'er', '▁pat', 'hu', 'tu', '▁ir', 'ken']])

### Training

In [67]:
data_lm = TextLMDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=taen_vocab, label_cols=['label'], text_cols=['sentence'])

/home/ec2-user/anaconda3/envs/temp_env/lib/python3.8/site-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


In [68]:
data_lm.show_batch()

idx,text
0,▁su bbu raj ▁ann e ▁ xxrep ▁4 ▁ . ▁in tha ▁padam ▁ve tri ▁ada ya ▁un a gal ukku ▁en n uda ya ▁val th ukk al . . . ▁x x bo s ▁kav un tar ▁the var . cha ar pa ak a ▁ver ri ▁per a ▁va a z h th th ukk al ▁ xxunk ▁x x bo s ▁ip po ▁in tha ▁tra
1,▁irukkum ▁unmai ya ana ▁naatak ak ▁ka a thal ▁vish ay ath th ai ▁cho n na al ▁at hu ▁cha ath ip ▁pat am ▁av lo o tha an ▁cha ar ▁poor a al e es ▁x x bo s ▁tamil ▁cinema ▁mon ster ▁surya ▁ xxrep ▁4 ▁ . ▁x x bo s ▁b g m ▁king ▁yu van ▁shankar ▁raja ▁x x bo s ▁pa kka ▁mas
2,? ▁tha lai var : s we et ▁sa ap ida ▁por om . . . ▁ver a ▁le vel . . . ▁x x bo s ▁at li ▁math i ri ▁mutt al u kal ▁kan du ▁padi kkat tum ▁e ppa di ▁padam ▁pud ikk anam ▁en tra th ▁x x bo s ▁vay as ukku ▁tha gu ndha ▁kel vi ▁ya ▁ke kka ▁sol lu nga ▁da
3,"▁x x bo s ▁kon g ku ▁cham u tha ay ath th inar ▁cha ar pa ak a ▁pat am ▁ver ri ▁per a ▁va a z h th th ukk al ▁x x bo s ▁ka agam ▁karai ndhu ▁ko odi ▁unn um , ▁mani dham ▁en num ▁mood ar ▁ko o dam ▁ko odi ▁ser dhu ▁pa ga i mai ▁kollum . . . ▁id il ▁yaar"
4,"▁nadi ku ran u ▁ka ko os ▁ar my ▁ko ov un anu nga ▁ xxunk ▁x x bo s ▁ ng k ▁was ▁in s pi red ▁to ▁ nt k ▁i ▁thi n k . ▁se e man ▁illamal ▁aras iyal um ▁illa i ▁ , arasi yal ▁padam um ▁illa i ▁nu ▁puri u dhu . . . ▁x x bo s ▁parv aiye e ▁ver a"


In [69]:
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.3, pretrained=False)

In [70]:
f'{path}/../models/models/best_model'

'./../models/models/best_model'

In [71]:
learn.load(f"{BASE_DIR}/temp/models/best_model", with_opt=True)

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (35139 items)
x: LMTextList
▁x x bo s ▁mo vi e ▁var a ▁le vel ▁la ▁eri ka ▁po gu th u,▁x x bo s ▁i ▁lo ve ▁aj ith ▁kumar ▁viv e gam ▁mo vi e ▁in ki ▁m j y ▁ bh t ▁achi ▁l gi,▁x x bo s ▁padam ▁nall a ▁com ed y ▁padam a ▁iru kum ▁pol aye . .,▁x x bo s ▁kar thi ck ▁su bbu raj ▁ann e ▁ xxrep ▁4 ▁ . ▁in tha ▁padam ▁ve tri ▁ada ya ▁un a gal ukku ▁en n uda ya ▁val th ukk al . . .,▁x x bo s ▁kav un tar ▁the var . cha ar pa ak a ▁ver ri ▁per a ▁va a z h th th ukk al ▁ xxunk
y: LMLabelList
,,,,
Path: .;

Valid: LabelList (4388 items)
x: LMTextList
▁x x bo s ▁han ds ome ▁hu n k ▁ke ri ▁va a ▁tha lai va a,▁x x bo s ▁the n ka a chi ▁ma av attam ▁na a ta ar ▁cham u tha ayam ▁cha ar pa ak a ▁va a z h th th ukk al,▁x x bo s ▁je ▁ vo us ▁ai me ▁br avo ▁po ur ▁cli p ▁de ▁mer de ▁qu e ▁j ▁ xxunk co u te ▁au ▁to il ette .,▁x x bo s ▁chir ap pu ▁melum ▁it hu ▁poon ra ▁pat ai ppu kal ▁mik a ▁ava chi yam,▁x x bo s ▁ver a ▁le vel ▁b g m ▁ . . s em ma ▁tr

In [72]:
learn.freeze()

In [73]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.044744,3.922262,0.326730,00:19


In [74]:
learn.save('fit_head', with_opt=True)

In [75]:
learn.load('fit_head', with_opt=True);

In [76]:
learn.unfreeze()

In [77]:
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.737872,3.653384,0.361356,00:20
1,3.440274,3.375876,0.398717,00:20
2,3.251985,3.262846,0.412751,00:20
3,3.197981,3.221164,0.418171,00:20
4,3.154185,3.215180,0.419078,00:20


In [78]:
learn.save('fine_tuned', with_opt=True)

In [79]:
learn.load('fine_tuned', with_opt=True);

In [80]:
learn.predict('Paka thana',n_words=10)

'Paka thana ▁it hu ▁naan ▁a ka ▁matt er ha ▁chinna ▁ra'

In [81]:
learn.save_encoder('fine_tuned_enc')

In [25]:
data_clas = TextClasDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=taen_vocab, bs=16, label_cols=['label'], text_cols=['sentence'])

/home/ec2-user/anaconda3/envs/temp_env/lib/python3.8/site-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


/home/ec2-user/anaconda3/envs/temp_env/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [83]:
data_clas.show_batch()

text,target
▁x x bo s ▁thi ro u pathi ▁ xxrep ▁10 ▁= ▁mann ai y um ▁penn ai y um ▁ka ak ka ▁vant ha ▁na aya ki ▁naatak a ▁ka ath alai ▁tho ol ur ikkum ▁nav een a ▁thi ro u pathi ▁pin j chil ▁pa z hu th tha a ▁ve m pi ▁tha an ▁va atum ▁nan j ch ai ▁chu vai th tha al ▁maranam,3
▁x x bo s ▁en ta a ▁e ena ▁pir avi kala a ▁avan ▁avan ▁ path th u ▁ma ach am ▁chu man th u ▁kash tap pattu ▁valar ath th u ▁tha an ▁aa ch ai ▁patta th tha ▁anup avi kka atti y um ▁than ▁ku z han th ai ▁ kku ▁the va iya ana tha ▁va ang ki ▁ku tu th th u ▁aa ch,3
▁x x bo s ▁anai th th u ▁cham ook a ▁ mak kalin ▁ka aman ts ▁ ith ai ▁pa ar kkum ▁po z hu th u ▁on ru ▁the li va ak a ▁puri kir ath u ▁oru ▁kuripp it ta ▁cham ook ath tha al ▁mar ra ▁anai th th u ▁cham ook amum ▁en tha ▁alav irku ▁pa ath ippu ▁at aint hu ▁irukki rat hu,0
▁x x bo s ▁ ith ula ▁enna ▁ka am e tin na a ▁oru ▁ pakk am ▁na ang ka ▁aan ta ▁param parai ▁pen ta ▁param para in nu ▁mee ch aiya i ▁mu rukk ur avan um ▁inno ru ▁ pakk am ▁at ang ka ▁maru ▁ ath th u ▁mee ru n nu ▁kam pu ▁chu th th ur avan um ▁50 varu sha ma a,3
▁x x bo s ▁vanni ya ▁kul a ▁k sh ath thi ri yar ▁( ak ni ▁kul a ▁k sh ath thi ri yar ▁ma kkal ▁ku tum path thil ▁ull a ▁vevveru ▁pattam ▁kont a ▁vanni yar kal ▁ xxunk van n iya ▁kav un tar ▁ xxunk van n iya ▁pat aiya atchi ▁ xxunk van n iya ▁na aya kar ▁ xxunk van n iya ▁va,0


In [84]:
data_clas.sanity_check()

/home/ec2-user/anaconda3/envs/temp_env/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [26]:
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.5)

In [86]:
learn.load_encoder('fine_tuned_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (35139 items)
x: TextList
▁x x bo s ▁mo vi e ▁var a ▁le vel ▁la ▁eri ka ▁po gu th u,▁x x bo s ▁i ▁lo ve ▁aj ith ▁kumar ▁viv e gam ▁mo vi e ▁in ki ▁m j y ▁ bh t ▁achi ▁l gi,▁x x bo s ▁padam ▁nall a ▁com ed y ▁padam a ▁iru kum ▁pol aye . .,▁x x bo s ▁kar thi ck ▁su bbu raj ▁ann e ▁ xxrep ▁4 ▁ . ▁in tha ▁padam ▁ve tri ▁ada ya ▁un a gal ukku ▁en n uda ya ▁val th ukk al . . .,▁x x bo s ▁kav un tar ▁the var . cha ar pa ak a ▁ver ri ▁per a ▁va a z h th th ukk al ▁ xxunk
y: CategoryList
0,4,0,0,0
Path: .;

Valid: LabelList (4388 items)
x: TextList
▁x x bo s ▁han ds ome ▁hu n k ▁ke ri ▁va a ▁tha lai va a,▁x x bo s ▁the n ka a chi ▁ma av attam ▁na a ta ar ▁cham u tha ayam ▁cha ar pa ak a ▁va a z h th th ukk al,▁x x bo s ▁je ▁ vo us ▁ai me ▁br avo ▁po ur ▁cli p ▁de ▁mer de ▁qu e ▁j ▁ xxunk co u te ▁au ▁to il ette .,▁x x bo s ▁chir ap pu ▁melum ▁it hu ▁poon ra ▁pat ai ppu kal ▁mik a ▁ava chi yam,▁x x bo s ▁ver a ▁le vel ▁b g m ▁ . . s em ma ▁tra

In [87]:
learn.freeze()

In [88]:
learn.loss_func.func

CrossEntropyLoss()

In [89]:
mcc = MatthewsCorreff()

In [90]:
learn.metrics = [mcc, accuracy]

In [91]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.841270,0.751917,0.263278,0.749772,00:56


In [92]:
learn.save('first-full')

In [93]:
learn.load('first-full');

In [94]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.750675,0.682011,0.381225,0.770283,00:55


/home/ec2-user/anaconda3/envs/temp_env/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [95]:
learn.save('second-full')

In [96]:
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3, callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='accuracy', name='final')])

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.658494,0.677764,0.412541,0.774385,01:00
1,0.657779,0.670823,0.418215,0.778943,00:55
2,0.656074,0.658323,0.427029,0.778259,00:55
3,0.615710,0.652915,0.439312,0.778259,00:56
4,0.521762,0.662419,0.452026,0.773473,01:02


/home/ec2-user/anaconda3/envs/temp_env/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Better model found at epoch 0 with accuracy value: 0.7743846774101257.


/home/ec2-user/anaconda3/envs/temp_env/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Better model found at epoch 1 with accuracy value: 0.7789425849914551.


/home/ec2-user/anaconda3/envs/temp_env/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/ec2-user/anaconda3/envs/temp_env/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to 

In [27]:
learn.load('final')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (35139 items)
x: TextList
▁x x bo s ▁mo vi e ▁var a ▁le vel ▁la ▁eri ka ▁po gu th u,▁x x bo s ▁i ▁lo ve ▁aj ith ▁kumar ▁viv e gam ▁mo vi e ▁in ki ▁m j y ▁ bh t ▁achi ▁l gi,▁x x bo s ▁padam ▁nall a ▁com ed y ▁padam a ▁iru kum ▁pol aye . .,▁x x bo s ▁kar thi ck ▁su bbu raj ▁ann e ▁ xxrep ▁4 ▁ . ▁in tha ▁padam ▁ve tri ▁ada ya ▁un a gal ukku ▁en n uda ya ▁val th ukk al . . .,▁x x bo s ▁kav un tar ▁the var . cha ar pa ak a ▁ver ri ▁per a ▁va a z h th th ukk al ▁ xxunk
y: CategoryList
0,4,0,0,0
Path: .;

Valid: LabelList (4388 items)
x: TextList
▁x x bo s ▁han ds ome ▁hu n k ▁ke ri ▁va a ▁tha lai va a,▁x x bo s ▁the n ka a chi ▁ma av attam ▁na a ta ar ▁cham u tha ayam ▁cha ar pa ak a ▁va a z h th th ukk al,▁x x bo s ▁je ▁ vo us ▁ai me ▁br avo ▁po ur ▁cli p ▁de ▁mer de ▁qu e ▁j ▁ xxunk co u te ▁au ▁to il ette .,▁x x bo s ▁chir ap pu ▁melum ▁it hu ▁poon ra ▁pat ai ppu kal ▁mik a ▁ava chi yam,▁x x bo s ▁ver a ▁le vel ▁b g m ▁ . . s em ma ▁tra

### Inference

In [28]:
df_test = df_valid.copy()
from sklearn.metrics import accuracy_score, matthews_corrcoef, f1_score
df_dict = {'query': list(df_test['sentence']), 'actual_label': list(df_test['label']), 'predicted_label': ['']*df_test.shape[0]}
all_nodes = list(set(df_valid['label']))
for node in all_nodes:
    df_dict[node] = ['']*df_test.shape[0]
    
i2c = {}
for key, value in learn.data.c2i.items():
    i2c[value] = key
    
df_result = pd.DataFrame(df_dict)
preds = learn.get_preds(ds_type=DatasetType.Valid, ordered=True)
nodes = []
predicted_labels = []
for index, row in df_result.iterrows():
    for node in all_nodes:
        df_result.loc[index, node] = preds[0][index][learn.data.c2i[node]].item()
    df_result.loc[index, 'predicted_label'] = i2c[np.argmax(preds[0][index]).data.item()]

df_result.head()

,query,actual_label,predicted_label,0,1,2,3,4,5
0,Handsome hunk keri vaa thalaivaa,0,0,0.954507,0.029217,0.006231,0.005025,0.002808,0.002211
1,thenkaachi maavattam naataar chamuthaayam chaa...,0,0,0.994352,0.001518,0.001340,0.002317,0.000093,0.000379
2,je vous aime bravo pour clip de merde que j éc...,4,4,0.369100,0.059537,0.077506,0.044610,0.438532,0.010714
3,chirappu melum ithu poonra pataippukal mika av...,0,0,0.955425,0.006989,0.008612,0.027626,0.000042,0.001306
4,Vera level BGM ..semma trailer. 🤞,0,0,0.932819,0.041869,0.003310,0.016839,0.002960,0.002203


In [29]:
import numpy
df_result['predicted_label'] = df_result['predicted_label'].astype(int)
df_result['actual_label'] = df_result['actual_label'].astype(int)

In [105]:
type(df_result['actual_label'].values[0]), type(df_result['predicted_label'].values[0])

(numpy.int64, numpy.int64)

In [30]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

0.7782588878760255

In [31]:
matthews_corrcoef(df_result['actual_label'], df_result['predicted_label'])


0.4163069605014294

In [32]:
f1_score(df_result['actual_label'], df_result['predicted_label'], average='weighted')

0.7335403148302653

In [33]:
precision_recall_fscore_support(df_result['actual_label'], df_result['predicted_label'], average='weighted')

/home/ec2-user/anaconda3/envs/temp_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.7269861245612425, 0.7782588878760255, 0.7335403148302653, None)

In [109]:
df_result['status'] = df_result['actual_label'] == df_result['predicted_label']
df_result[df_result['status'] == False]

,query,actual_label,predicted_label,0,1,2,3,4,5,status
17,dei YENDA ungalukku inthe illatha Vella Surya ...,2,0,0.354947,0.263264,0.204207,0.132686,0.008453,0.036443,False
21,thala innum ipdi full white evvalavu naal nadi...,2,0,0.887194,0.032376,0.036545,0.037426,0.000109,0.006349,False
27,Deii ponga da tamil naatla rape crime increase...,3,0,0.467927,0.080725,0.038836,0.393410,0.000704,0.018398,False
36,Comment la en da picha edukuringa... pichakara...,3,1,0.049569,0.548068,0.007857,0.331396,0.001707,0.061404,False
40,Karthi fanslike Kaithi waiting but bigil maran...,2,0,0.395608,0.056239,0.394572,0.133157,0.003005,0.017420,False
...,...,...,...,...,...,...,...,...,...,...
4369,Ivarukkku eppodhum thalaivar kalaigner lightaa...,2,0,0.369844,0.208622,0.288573,0.035840,0.075511,0.021610,False
4372,Trailer Nala irukanu oru than comment pandranu...,1,0,0.812631,0.129006,0.018768,0.032908,0.000238,0.006449,False
4378,Wigpathy Visay na Padam Flop than ithula Kabal...,2,3,0.035182,0.140930,0.071759,0.697300,0.003162,0.051667,False
4380,Vikram ella styleum set aaguthu.. Namba moonji...,2,0,0.426659,0.214986,0.234326,0.094319,0.004637,0.025073,False


In [ ]:
df_result.to_csv(f"{BASE_DIR}/tamil_valid_results_umlfit.csv", index=False)

In [115]:
df_test = pd.read_csv(f"{BASE_DIR}/tamil_offensive_full_test_transliterated.csv")
print(df_test.shape)
df_test.dropna(inplace=True)
print(df_test.shape)

(4392, 1)
(4392, 1)


In [116]:
df_test

,sentence
0,14.12.2018epo trailer pathutu irken ...Semay...
1,Paka thana poro movie la Enna irukunu
2,“U kena tunggu lebih lama lagi untuk tahu saya...
3,Suriya anna vera level anna mass
4,suma kaththaatha da sound over a pooda kudaath...
...,...
4387,mannu ponnu rentume onnu athula evan kaiya vac...
4388,Babu mele ko ye song sunke kuch yesa feel hua ...
4389,asuran= aadukalam+pudupettai+ wada chennai..ye...
4390,Vijay's all movies look like same.


In [138]:
df_dict = {'sentence': list(df_test['sentence']), 'label': ['']*df_test.shape[0]}
all_nodes = list(set(df_train['label']))
for node in all_nodes:
    df_dict[node] = ['']*df_test.shape[0]
    
i2c = {}
for key, value in learn.data.c2i.items():
    i2c[value] = key

df_result = pd.DataFrame(df_dict)


In [139]:
df_result

,sentence,label,0,1,2,3,4,5
0,14.12.2018epo trailer pathutu irken ...Semay...,,,,,,,
1,Paka thana poro movie la Enna irukunu,,,,,,,
2,“U kena tunggu lebih lama lagi untuk tahu saya...,,,,,,,
3,Suriya anna vera level anna mass,,,,,,,
4,suma kaththaatha da sound over a pooda kudaath...,,,,,,,
...,...,...,...,...,...,...,...,...
4387,mannu ponnu rentume onnu athula evan kaiya vac...,,,,,,,
4388,Babu mele ko ye song sunke kuch yesa feel hua ...,,,,,,,
4389,asuran= aadukalam+pudupettai+ wada chennai..ye...,,,,,,,
4390,Vijay's all movies look like same.,,,,,,,


In [142]:
for index, row in df_result.iterrows():
    result = learn.predict(row['sentence'])
    prob =  result[2]
    for node in all_nodes:
        df_result.loc[index, node] = prob[learn.data.c2i[node]].item()
    df_result.loc[index,'label'] = i2c[np.argmax(prob).data.item()]
    df_result.loc[index, 'label'] = result[1].item()
df_result.head()

,sentence,label,0,1,2,3,4,5
0,14.12.2018epo trailer pathutu irken ...Semay...,0,0.737698,0.207027,0.018546,0.027930,0.001813,0.006987
1,Paka thana poro movie la Enna irukunu,0,0.596777,0.275670,0.031983,0.070101,0.000985,0.024484
2,“U kena tunggu lebih lama lagi untuk tahu saya...,4,0.005738,0.000338,0.000252,0.000648,0.992902,0.000122
3,Suriya anna vera level anna mass,0,0.995216,0.000994,0.000411,0.002299,0.000804,0.000275
4,suma kaththaatha da sound over a pooda kudaath...,2,0.079670,0.115144,0.590439,0.189508,0.001885,0.023356


In [143]:
df_result['label'].value_counts()

0    3812
1     181
4     158
2     146
3      95
Name: label, dtype: int64

In [46]:
learn.data.c2i, i2c

({0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}, {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5})

In [144]:
df_result.to_csv(f"{BASE_DIR}/tamil_test_results_ulmfit.csv", index=False)

In [145]:
df_result.shape

(4392, 8)